In [50]:
import pandas as pd
from string import ascii_uppercase as alpha
import pickle
from bs4 import BeautifulSoup
import requests
import time

In [24]:
all_tables=pd.read_html('https://web.archive.org/web/20221002180521/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')

In [25]:
dict_tables={}
for letter,i in zip(alpha,range(11,67,7)):
    df=all_tables[i]
    df.rename(columns={df.columns[1]:'Team'},inplace=True)
    dict_tables[f"Cluster:{letter}"]=df

To csv Fixture..

In [26]:
df=pd.concat(dict_tables)
df.to_csv('Fixture_2022.csv',index=False)

Starting to filter the page

In [28]:
years=[1930,1934,1938,1950,1954,1958,1962,1966,1970,1974,1978,1982,1986,1990,1994,1998,2002,2006,2010,2014,2018]

making a request

In [29]:
web='https://en.wikipedia.org/wiki/2014_FIFA_World_Cup'
response=requests.get(web)

Loading text with Beatifulsoup

In [30]:
content=response.text
soup=BeautifulSoup(content,'lxml')

Filtering rows on matches

In [31]:
matches=soup.findAll('div',class_='footballbox')


Filtering Matches and adding to list for dataframe

In [32]:
home=[]
score=[]
away=[]
for match in matches:
    home.append(match.find('th',class_='fhome').get_text())
    score.append(match.find('th',class_='fscore').get_text())
    away.append(match.find('th',class_='faway').get_text())
#the elements have already been added to the lists

Creating the dictionary previusly for the Dataframe

In [33]:
dict_football={'home':home ,'score':score ,'Away':away}
df_football=pd.DataFrame(dict_football)
df_football['year']='2014'

To make the work easier, we are going to use a function, that does exactly the same as the previous thing..

In [34]:
def get_matches(year):
    web=f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    response=requests.get(web)
    content=response.text
    soup=BeautifulSoup(content,'lxml')
    matches=soup.findAll('div',class_='footballbox')
    home=[]
    score=[]
    away=[]
    for match in matches:
        home.append(match.find('th',class_='fhome').get_text())
        score.append(match.find('th',class_='fscore').get_text())
        away.append(match.find('th',class_='faway').get_text())
    #the elements have already been added to the lists
    dict_football={'home':home ,'score':score ,'Away':away}
    df_football=pd.DataFrame(dict_football)
    df_football['year']=year
    return df_football

Let's try it..

In [35]:
#get_matches(1930)

we are going to save all the dataframes in a list

In [36]:
fifa=[get_matches(year)for year in years]
df_fifa=pd.concat(fifa,ignore_index=True)

To csv...

In [37]:
df_fifa.to_csv('Fifa_worldcup_historical_data1.csv',index=False)

Because in certain world cups, with beautifulsoup it is not possible to obtain the data, so we will use selenium, for this especific cups

Using selenium to improve the process for collect data

In [53]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service



Importing the chromedriver

In [39]:
service = Service('chromedriver_linux64')

Creating driver to interacts to web page

In [40]:
driver=webdriver.Chrome(service=service)
web='https://en.wikipedia.org/wiki/1982_FIFA_World_Cup'
driver.get(web)


Getting the xpaht of the matches

In [41]:
matches=driver.find_elements(by='xpath',value="//th[@class='fhome']/..")

Extracting the information from the score, home, away

In [47]:
home=[]
score=[]
away=[]
for match in matches:
    home.append(match.find_element(by='xpath',value="./th[1]").text)
    score.append(match.find_element(by='xpath',value="./th[2]").text)
    away.append(match.find_element(by='xpath',value="./th[3]").text)

Creating a Dataframe..

In [49]:
dict_football2={'home':home,'score':score,'away':away}
df_history_worldcup=pd.DataFrame(dict_football2)
df_history_worldcup.to_csv('World_cup_1982.csv',index=False)


Now we are going to apply all that in a function that goes through all the world cups and prints it in a csv....

In [56]:
def get_all_dataworldcup(year):
    service = Service('chromedriver_linux64')
    driver=webdriver.Chrome(service=service)
    web=f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    driver.get(web)
    matches=driver.find_elements(by='xpath',value="//th[@class='fhome']/..")
    home=[]
    score=[]
    away=[]
    for match in matches:
        home.append(match.find_element(by='xpath',value="./th[1]").text)
        score.append(match.find_element(by='xpath',value="./th[2]").text)
        away.append(match.find_element(by='xpath',value="./th[3]").text)
    dict_football2={'home':home,'score':score,'away':away}
    df_history_worldcup=pd.DataFrame(dict_football2)
    df_history_worldcup['year']=year
    time.sleep(2)
    return df_history_worldcup
#this element is out of a function because
fifa2=[get_all_dataworldcup(year) for year in years]
df_fifa2=pd.concat(fifa2,ignore_index=True)
df_fifa2.to_csv('Fifa_worldcup_missing_data.csv',index=False)
driver.quit()